In [13]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("data/CraigVenter_2008.sph")
print(result["text"])

/home/alex/anaconda3/envs/audio-sifter/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alex/anaconda3/envs/audio-sifter/lib/python3.10/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 You know, I've talked about some of these projects before about the human genome and what that might mean, and discovering new sets of genes. We're actually starting at a new point. We've been digitizing biology, and now we're trying to go from that digital code into a new phase of biology with designing and synthesizing life. So we've always been trying to ask big questions. What is life is something that I think many biologists have been trying to understand at various levels. We've tried various approaches, pairing it down to minimal components. We've been digitizing it now for almost 20 years when we sequenced the human genome. It was going from the analog world of biology into the digital world of the computer. Now we're trying to ask, can we regenerate life, or can we create new life out of this digital universe? This is the map of a small organism, microplasma genitalium. That has the smallest genome for a species that can self-replicate in the laboratory. We've been trying to 

In [14]:
text_segments = result["segments"]
print(text_segments[0])

{'id': 0, 'seek': 0, 'start': 0.0, 'end': 20.400000000000002, 'text': " You know, I've talked about some of these projects before about the human genome and what", 'tokens': [50364, 509, 458, 11, 286, 600, 2825, 466, 512, 295, 613, 4455, 949, 466, 264, 1952, 21953, 293, 437, 51384, 51384, 300, 1062, 914, 11, 293, 24773, 777, 6352, 295, 14424, 13, 51620, 51620], 'temperature': 0.0, 'avg_logprob': -0.3250291006905692, 'compression_ratio': 1.2589285714285714, 'no_speech_prob': 0.1914173662662506}


In [15]:
import requests
import uuid

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_schema = """{
    'title': 'talk',
    'type': 'object',
    'properties': {
        'id': { 'type': 'string', 'description': 'The id of a talk' },
        'start': {'type': 'float', 'description': 'Start of audio segment'},
        'end': {'type': 'float', 'description': 'End of audio segment'},
        'year': {'type': 'int', 'description': 'Year the talk was held'},
        'speaker': { 'type': 'string', 'description': 'The name of the speaker' },
        'content': { 'type': 'string', 'description': 'The content of a talk' },
        'talk-type': { 'type': 'string', 'description': 'The host that published this talk' }
    }
}"""

In [16]:
# create index
data = '{"name": "talk",\n  "txt_field": "content",\n  "bucket_field": "talk-type",\n  "json_schema": "{json_schema}"}'

response = requests.post('http://127.0.0.1:8000/idx', headers=headers, data=data)

In [17]:
# index items
for segment in text_segments:

    json_data = {
        'id': str(uuid.uuid4()),
        'start': segment['start'],
        'end': segment['start'],
        'year': 2008,
        'speaker': 'Craig Venter',
        'content': segment['text'],
        'talk-type': 'ted',
    }   

    response = requests.put('http://127.0.0.1:8000/item/talk', headers=headers, json=json_data)


In [18]:
json_data = {
    'query': 'Chromosome transplant',
    'bucket': 'ted',
    'limit': 200,
}

response = requests.post('http://127.0.0.1:8000/qry/talk', headers=headers, json=json_data)

In [28]:
import json

r = json.loads(response.content)
r

[{'item': {'id': 'ad484243-bd2f-419c-b7d6-411b597d3fcd',
   'talk-type': 'ted',
   'content': ' transplant of a chromosome from one cell to another and activate it. We purified a chromosome',
   'year': 2008,
   'start': 515.48,
   'end': 515.48,
   'speaker': 'Craig Venter'},
  'score': 0.7013537883758545},
 {'item': {'id': '516bf2ad-acdc-4c08-b1d3-8bcdaa061628',
   'talk-type': 'ted',
   'content': ' the chromosome is integrated into the cell. We recently showed that we can do a complete',
   'year': 2008,
   'start': 509.64000000000004,
   'end': 509.64000000000004,
   'speaker': 'Craig Venter'},
  'score': 0.5927460193634033},
 {'item': {'id': '15926ce7-6d26-4b66-8f65-c119a9836ecf',
   'talk-type': 'ted',
   'content': ' we put in does. It got expressed and it recognized the other chromosome as foreign material',
   'year': 2008,
   'start': 581.0,
   'end': 581.0,
   'speaker': 'Craig Venter'},
  'score': 0.5793034434318542},
 {'item': {'id': '25d5691d-d522-477f-a83f-36897df24ff9'